### Import libraries

In [259]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import csv

### Define RNN model

In [260]:
class RNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=2, batch_first=True, dropout=0.3)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # last timestep
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out  # raw logits

### Test run

In [261]:
model = RNN_Model(input_size=1, hidden_size=64)
x = torch.randn(64, 10, 1)  # batch=5, seq_len=7, input_size=10
output = model(x)
print("Done" if output.shape == (64, 1) else "Failed")

Done


### Data loading

In [262]:
class SequenceDataset(Dataset):
    def __init__(self, csv_file, max_len):
        self.sequences = []
        self.labels = []
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            next(reader)  # skip header
            for row in reader:
                label = int(row[-1])
                seq = list(map(int, row[:-1]))
                self.sequences.append(seq)
                self.labels.append(label)
        self.max_len = max_len


    def __len__(self):
        return len(self.sequences)


    def __getitem__(self, idx):
        seq = self.sequences[idx]
        # Normalize values to [0, 1], keep padding as -1
        seq = [(v if v != 0 else -1.0) for v in seq]
        seq_tensor = torch.tensor(seq, dtype=torch.float32).unsqueeze(-1)  # shape: [seq_len, 1]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return seq_tensor, label

In [263]:
dataset = SequenceDataset('order_dependent_sequence_dataset.csv', max_len=10)

### "dataset" instance testing

In [264]:
seq, label = dataset[1]
print("Sequence:", seq)
print("Label:", label)

Sequence: tensor([[-1.],
        [ 6.],
        [ 8.],
        [ 5.],
        [-1.],
        [-1.],
        [ 5.],
        [ 5.],
        [ 3.],
        [-1.]])
Label: tensor(0.)


### Training Loop

In [265]:
batch_size = 64
hidden_size = 64
lr = 0.001
num_epochs = 200

In [266]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [267]:
for sequences, labels in dataloader:
    sequences, labels = sequences.to(torch.device("cpu")), labels.to(torch.device("cpu")).unsqueeze(1)
    print("Batch sequences shape:", sequences.shape)
    print("Batch labels shape:", labels.shape)
    break

Batch sequences shape: torch.Size([64, 10, 1])
Batch labels shape: torch.Size([64, 1])


### Instantiate model, loss, optimizer

In [268]:
model = RNN_Model(input_size=1, hidden_size=hidden_size)
criterion = nn.BCEWithLogitsLoss()  # combines sigmoid + BCE loss
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [269]:
device = torch.device("cuda")
model.to(device)

RNN_Model(
  (rnn): RNN(1, 64, num_layers=2, batch_first=True, dropout=0.3)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [270]:
for epoch in range(1, num_epochs):
    model.train()
    total_loss = 0
    for sequences, labels in dataloader:
        sequences, labels = sequences.to(device), labels.to(device).unsqueeze(1)

        outputs = model(sequences)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)

    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}")
print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [10/200], Loss: 0.4387
Epoch [20/200], Loss: 0.4292
Epoch [30/200], Loss: 0.3608
Epoch [40/200], Loss: 0.2671
Epoch [50/200], Loss: 0.1673
Epoch [60/200], Loss: 0.0698
Epoch [70/200], Loss: 0.0458
Epoch [80/200], Loss: 0.0177
Epoch [90/200], Loss: 0.0107
Epoch [100/200], Loss: 0.0075
Epoch [110/200], Loss: 0.0066
Epoch [120/200], Loss: 0.0049
Epoch [130/200], Loss: 0.0043
Epoch [140/200], Loss: 0.0097
Epoch [150/200], Loss: 0.0136
Epoch [160/200], Loss: 0.0046
Epoch [170/200], Loss: 0.0542
Epoch [180/200], Loss: 0.0016
Epoch [190/200], Loss: 0.0032
Epoch [199/200], Loss: 0.0031


In [281]:
manual_sequence = [7, 2, 7, 8, 7, 3]  # example input, change as needed

# Pad or truncate the sequence to max_len
padded = manual_sequence[:10] + [-1] * (10 - len(manual_sequence))
# Convert PAD_VALUE to -1.0, others to float
input_seq = [float(v) if v != -1 else -1.0 for v in padded]
input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)  # shape: [1, max_len, 1]

model.eval()
with torch.no_grad():
    logits = model(input_tensor)
    prob = torch.sigmoid(logits)
    
    if prob.item() > 0.5:
        print("Predicted label: 1")
    else:
        print("Predicted label: 0")

Predicted label: 0


In [282]:
def get_label_from_sequence(sequence):
    found_3 = False
    label = 0
    for num in sequence:
        if num == 3:
            found_3 = True
        elif num == 7 and found_3:
            label = 1
            break
    return label

# Example usage:
print("Calculated label: ", get_label_from_sequence(manual_sequence))  # Output: 1

Calculated label:  0
